### Monotone frequency implementation

In [16]:
import pyaudio
import wave
import numpy as np
p = pyaudio.PyAudio()
SAMPLING_FREQ = 44100  # sampling rate, Hz, must be integer
DURATION = 3  # in seconds, may be float
CHANNELS = 2
FORMAT = pyaudio.paFloat32
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=SAMPLING_FREQ,
                output=True)

In [17]:
def write2stream(fs, duration, stream, part, relative_dist, VOL_SCALE=5/50, MAX_RELATIVE_DIST=4, NEAR_RELATIVE_DIST=0.3, MAX_FREQ=800, MIN_FREQ=400):

    if not relative_dist:
        samples = np.zeros(CHANNELS * DURATION)
    
    else:
        side = part[0] # l or r
        mask = [side == "l", side == "r"] * int((fs*duration)) # generate mask to fire 1 channel, clear other

        if relative_dist < NEAR_RELATIVE_DIST:
            freq = 1200 # TODO: refine `correct` signal
        else:
            freq = MAX_FREQ - (relative_dist / MAX_RELATIVE_DIST) * (MAX_FREQ - MIN_FREQ)

        time_samples = np.arange(fs * 2*duration) * freq / fs
        samples = (np.sin(2 * np.pi * time_samples*mask)).astype(np.float32)
        samples *= VOL_SCALE

    output_bytes = samples.tobytes()
    
    with wave.open("test.wav", "wb") as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(SAMPLING_FREQ)
        wf.writeframes(output_bytes)
    stream.write(output_bytes)

In [18]:
write2stream(fs=SAMPLING_FREQ, duration=DURATION, stream=stream, part="l_foot", relative_dist=10)

### Variable Freq implementation

In [1]:
import pyaudio
import wave
import numpy as np

In [8]:
p = pyaudio.PyAudio()
FS = 44100  # sampling rate, Hz, must be integer
CHANNELS = 2
FORMAT = pyaudio.paFloat32
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=FS,
                output=True)

def write2stream(part, relative_dist, stream, MAX_DIST=7, MIN_DIST=0.6, DURATION=0.1, SOUND_ARGS=(333, 450), VOL_SCALE=0.2, FS=44100):

    if relative_dist != None and part != None:
        side = part[0]
        if relative_dist > MIN_DIST:
            if relative_dist > MAX_DIST:
                relative_dist = MAX_DIST
            freq = int(SOUND_ARGS[0]/relative_dist + SOUND_ARGS[1])
            channel_mask = [side == "l", side == "r"] * int((FS*DURATION)) # generate mask to fire 1 channel, clear other
            time_samples = np.arange(FS * 2*DURATION) * freq / FS
            samples = (np.sin(2 * np.pi * time_samples*channel_mask)).astype(np.float32)
        
        else:
            duration = 1
            LOW_FREQ, HIGH_FREQ = 300, 700
            channel_mask = [side == "l", side == "r"] * int((FS*duration)) # generate mask to fire 1 channel, clear other
            time_samples1 = np.arange(FS * 2*(1/4*duration)) * LOW_FREQ / FS
            time_samples2 = np.arange(FS * 2*(1/4*duration)) * HIGH_FREQ / FS
            time_samples = np.tile(np.concatenate((time_samples1, time_samples2), axis=0), 2)
            samples = (np.sin(2 * np.pi * time_samples*channel_mask)).astype(np.float32)
            ones = np.ones(int(0.2*len(samples)))
            zeros = np.zeros(int(0.05*len(samples)))
            audio_mask = np.tile(np.concatenate((ones, zeros), axis=0), 4)
        
        samples *= VOL_SCALE
        output_bytes = samples.tobytes()
        stream.write(output_bytes)


In [9]:
while True:
    write2stream(part="l_foot", relative_dist=1, stream=stream)

KeyboardInterrupt: 

In [19]:
def beep(part, stream, DURATION=1, FREQ_RANGE=(300,700), VOL_SCALE=0.2, FS=44100):

    side = part[0]
    (LOW_FREQ, HIGH_FREQ) = FREQ_RANGE
    channel_mask = [side == "l", side == "r"] * int((FS*DURATION)) # generate mask to fire 1 channel, clear other
    time_samples1 = np.arange(FS * 2*(1/4*DURATION)) * LOW_FREQ / FS
    time_samples2 = np.arange(FS * 2*(1/4*DURATION)) * HIGH_FREQ / FS
    time_samples = np.tile(np.concatenate((time_samples1, time_samples2), axis=0), 2)
    samples = (np.sin(2 * np.pi * time_samples*channel_mask)).astype(np.float32)
    ones = np.ones(int(0.2*len(samples)))
    zeros = np.zeros(int(0.05*len(samples)))
    audio_mask = np.tile(np.concatenate((ones, zeros), axis=0), 4)
    samples *= audio_mask
        
    samples *= VOL_SCALE
    output_bytes = samples.tobytes()
    stream.write(output_bytes)

In [20]:
beep(part="r_foot", stream=stream)